* python 실행 경로 확인 
    - python 가상 환경을 사용 중이라면 가상 환경의 path 가 맞는지 확인한다. 


In [7]:
%gui qt5
%matplotlib inline

import matplotlib.pyplot as plt

import sys
print(sys.executable)

ModuleNotFoundError: No module named 'matplotlib'

## 1. 객체 생성

In [4]:
from kw_condition import KiwoomOpenApiPlus
from PySide2.QtWidgets import QApplication

myApp = None
if isinstance(QApplication.instance(), type(None)):
    print('make instance')
    myApp = QApplication([])
else:
    print("use already")
    myApp = QApplication.instance()

kw_obj = KiwoomOpenApiPlus()

kw_condition.backend.KiwoomOpenApiPlus <module 'kw_condition.backend.KiwoomOpenApiPlus' from 'D:\\1git\\kw_condition\\kw_condition\\backend\\KiwoomOpenApiPlus.py'>
kw_condition.backend.KiwoomOpenApiPlus <module 'kw_condition.backend.KiwoomOpenApiPlus' from 'D:\\1git\\kw_condition\\kw_condition\\backend\\KiwoomOpenApiPlus.py'>
use already


## 2. 서버접속
* 자동로그인이 설정되어 있는 경우 로그인이 자동으로 처리 되며, 그렇지 않은 경우 팝업 창에서 수동으로 아이디, 비밀번호 정보를 입력해야함 
- TODO
    * 자동 로그인 설정되지 않은 상태서 로그인 정보를 외부에서 입력한 경우 매크로로 자동처리 할수 있도록 해야함 pywinauto

In [5]:
import time
kw_obj.tryConnect()

loop_count = 0
while kw_obj.isConnected() == False:
    loop_count = loop_count + 1
    time.sleep(1)
    QApplication.processEvents()
    if( loop_count > 30 ) :
        print('connect failed!')
        break



* 11:49:26.863261 initStateEntered 
* 11:49:26.870238 disconnectedStateEntered 
* 11:49:26.918888 tryConnect 
* 11:49:30.146662 _OnEventConnect 0
* 11:49:30.154691 connectedStateEntered 
account count: 1, acc_num: 4175351811;, user id: coreanq, user_name: 박국철, keyboard_boan: 1, firewall: 2
* 11:49:30.162923 connectedStateEntered account list ['4175351811']


## 3. 서버 접속 상태 확인 

In [6]:
kw_obj.isConnected()

True

## 4. TR 요청 예시 

아래 처럼 직접 TR 요청에 필요한 입력값을 설정해 요청하고, 이후 들어오는 이벤트 또한 직접 처리해주는 방식으로 사용할 수 있다.

In [1]:
rqname = '주식기본정보요청'
trcode = 'opt10001'

screen_no = '0001'  # 화면번호, 0000 을 제외한 4자리 숫자 임의로 지정, None 의 경우 내부적으로 화면번호 자동할당
inputs = {'종목코드': '005930'}

kw_obj.transaction(rqname, trcode, screen_no, inputs )


NameError: name 'code' is not defined